In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
file_name = 'jamb_exam_results.csv'

In [8]:
if os.path.exists(file_name):
    pass
else:
    !wget https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv

--2024-10-31 15:54:34--  https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv [following]
--2024-10-31 15:54:43--  https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391501 (382K) [text/plain]
Saving to: 'jamb_exam_results.csv'

     0K .......... .......... .......... .......... .......... 13%  199K 2s
    50K .......... .......... .......... .......... ....

In [9]:
df = pd.read_csv(file_name)

In [10]:
df.columns = df.columns.str.lower().str.replace(' ',_)

In [12]:
df.columns

Index(['jamb_score', 'study_hours_per_week', 'attendance_rate',
       'teacher_quality', 'distance_to_school', 'school_type',
       'school_location', 'extra_tutorials', 'access_to_learning_materials',
       'parent_involvement', 'it_knowledge', 'student_id', 'age', 'gender',
       'socioeconomic_status', 'parent_education_level',
       'assignments_completed'],
      dtype='object')

In [14]:
df = df.drop(columns='student_id').fillna(0)

In [15]:
df.isnull().sum()

jamb_score                      0
study_hours_per_week            0
attendance_rate                 0
teacher_quality                 0
distance_to_school              0
school_type                     0
school_location                 0
extra_tutorials                 0
access_to_learning_materials    0
parent_involvement              0
it_knowledge                    0
age                             0
gender                          0
socioeconomic_status            0
parent_education_level          0
assignments_completed           0
dtype: int64

In [16]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

In [18]:
y = df['jamb_score']
X = df.drop(columns='jamb_score').copy()

In [20]:
len(df)

5000

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.25,random_state=1)
len(X_train),len(X_test),len(X_val)

(3000, 1000, 1000)

In [22]:
X_train_dict = X_train.to_dict(orient='records') 
X_train_dict

[{'study_hours_per_week': 20,
  'attendance_rate': 72,
  'teacher_quality': 3,
  'distance_to_school': 4.4,
  'school_type': 'Public',
  'school_location': 'Urban',
  'extra_tutorials': 'No',
  'access_to_learning_materials': 'Yes',
  'parent_involvement': 'Medium',
  'it_knowledge': 'Low',
  'age': 21,
  'gender': 'Female',
  'socioeconomic_status': 'Low',
  'parent_education_level': 'None',
  'assignments_completed': 3},
 {'study_hours_per_week': 11,
  'attendance_rate': 80,
  'teacher_quality': 2,
  'distance_to_school': 3.3,
  'school_type': 'Public',
  'school_location': 'Urban',
  'extra_tutorials': 'Yes',
  'access_to_learning_materials': 'Yes',
  'parent_involvement': 'Medium',
  'it_knowledge': 'High',
  'age': 22,
  'gender': 'Female',
  'socioeconomic_status': 'Medium',
  'parent_education_level': 'Secondary',
  'assignments_completed': 1},
 {'study_hours_per_week': 31,
  'attendance_rate': 82,
  'teacher_quality': 1,
  'distance_to_school': 8.3,
  'school_type': 'Public',
 

In [35]:
dv = DictVectorizer(sparse=True)
X_train_vect = dv.fit_transform(X_train_dict)
X_train_vect

<3000x29 sparse matrix of type '<class 'numpy.float64'>'
	with 45000 stored elements in Compressed Sparse Row format>

In [36]:
X_test_vect = dv.transform(X_test.to_dict(orient='records'))
X_val_vect = dv.transform(X_val.to_dict(orient='records'))

In [37]:
X_test_vect

<1000x29 sparse matrix of type '<class 'numpy.float64'>'
	with 15000 stored elements in Compressed Sparse Row format>

In [42]:
X_val_vect


<1000x29 sparse matrix of type '<class 'numpy.float64'>'
	with 15000 stored elements in Compressed Sparse Row format>

In [46]:
from sklearn.tree import DecisionTreeRegressor, export_text

In [44]:
dtr = DecisionTreeRegressor(max_depth=1)
dtr.fit(X_train_vect,y_train)

DecisionTreeRegressor(max_depth=1)

In [45]:
dtr

DecisionTreeRegressor(max_depth=1)

In [48]:
tree_text = export_text(dtr, feature_names=dv.feature_names_)
print(tree_text)

|--- study_hours_per_week <= 18.50
|   |--- value: [155.24]
|--- study_hours_per_week >  18.50
|   |--- value: [188.59]



In [49]:
from sklearn.metrics import root_mean_squared_error, auc

In [52]:
from sklearn.ensemble import RandomForestRegressor

In [54]:
rfr = RandomForestRegressor(n_estimators = 10,random_state=1,n_jobs=-1)
rfr.fit(X_train_vect,y_train)
y_pred = rfr.predict(X_val_vect)
root_mean_squared_error(y_val,y_pred)

41.82546234054084

In [58]:
from tqdm import tqdm

In [63]:
estimators = [x for x in range(10,201,10)]
rmse_val = dict()
for nest in tqdm(estimators):
    rfr = RandomForestRegressor(n_estimators = nest,random_state=1,n_jobs=-1)
    rfr.fit(X_train_vect,y_train)
    y_pred = rfr.predict(X_val_vect)
    rmse = root_mean_squared_error(y_val,y_pred)
    rmse_val[nest] = rmse
rmse_val

100%|██████████| 20/20 [00:18<00:00,  1.11it/s]


{10: 41.82546234054084,
 20: 41.19602596125019,
 30: 40.856505465946164,
 40: 40.66637198749109,
 50: 40.678118442228865,
 60: 40.6039010304292,
 70: 40.53460449582126,
 80: 40.403939657461,
 90: 40.376522525281544,
 100: 40.43028290774132,
 110: 40.47245412796827,
 120: 40.52150253517672,
 130: 40.529568505653224,
 140: 40.4652468105506,
 150: 40.43557058443579,
 160: 40.46153425052305,
 170: 40.47871275067374,
 180: 40.4824103492515,
 190: 40.46830275530142,
 200: 40.454002673715735}

In [66]:
dephts = [10,15,20,25]

rmse_val = dict()
for max_depth in tqdm(dephts):
    rmse_list = []
    for nest in estimators:
        rfr = RandomForestRegressor(max_depth=max_depth,n_estimators = nest,random_state=1,n_jobs=-1)
        rfr.fit(X_train_vect,y_train)
        y_pred = rfr.predict(X_val_vect)
        rmse = root_mean_squared_error(y_val,y_pred)
        rmse_list.append(rmse)
    rmse_val[max_depth] = rmse_list
rmse_val

100%|██████████| 4/4 [01:03<00:00, 15.76s/it]


{10: [41.07251191004754,
  40.85349421020777,
  40.62338184364773,
  40.334001530035884,
  40.31986975658787,
  40.2617749736961,
  40.28085644907353,
  40.22801311289593,
  40.173268717278695,
  40.22583079255509,
  40.2518427269761,
  40.27616485721727,
  40.29349764167316,
  40.25771096743785,
  40.271261216300225,
  40.309501428995716,
  40.3241900774062,
  40.332298664520096,
  40.31792740168356,
  40.29306087455316],
 15: [41.863590922930534,
  41.17897179483894,
  40.91056612460649,
  40.80042967430943,
  40.75808374135911,
  40.70300523378798,
  40.63858575886564,
  40.49309526718576,
  40.43891936377008,
  40.425098853515394,
  40.47350596361154,
  40.506463980951494,
  40.51099438996265,
  40.4460090602511,
  40.42810557174929,
  40.4467729657673,
  40.46964652913519,
  40.46453734649347,
  40.4374577706849,
  40.40148907117545],
 20: [41.95537038873333,
  41.34435231581997,
  41.009537468633084,
  40.79765148495233,
  40.70601448302282,
  40.65511050557274,
  40.625307470748

In [69]:
max_depth_dict = {}
for key,val in rmse_val.items():
    max_depth_dict[key] = sum(val)/len(val)

max_depth_dict

{10: 40.36502295763948,
 15: 40.63976646924758,
 20: 40.68456298423326,
 25: 40.61136228599571}

In [74]:
rfr2 = RandomForestRegressor(max_depth=20,
                            n_estimators = 10,
                            random_state=1,
                            n_jobs = -1)
rfr2.fit(X_train_vect,y_train)
y_pred = rfr2.predict(X_val_vect)
root_mean_squared_error(y_val,y_pred)

41.95537038873333

In [85]:
feature_scores = pd.Series(rfr2.feature_importances_, index = dv.feature_names_).sort_values(ascending=False)

In [90]:
feature_scores.index[0]

'study_hours_per_week'

In [97]:
import xgboost as xgb

In [113]:
#creating dmatrix for training and validation

dtrain = xgb.DMatrix(X_train_vect, label = y_train,feature_names = dv.feature_names_,enable_categorical=True)
dval = xgb.DMatrix(X_val_vect, label = y_val,feature_names = dv.feature_names_,enable_categorical=True)

In [110]:
dtrain


In [92]:
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

In [123]:
xgb_params2 = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

In [118]:
#Watchlist -> To get an idea of how AUC changes as the number of trees grows, we can use a watchlist — a built-in feature in XGBoost for monitoring model performance.

watchlist = [(dtrain,'train'), (dval,'val')]

In [119]:
xgbr = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round = 100,
    evals = watchlist
)

[0]	train-rmse:42.69384	val-rmse:44.89114
[1]	train-rmse:39.83326	val-rmse:43.07010
[2]	train-rmse:37.94542	val-rmse:42.00332
[3]	train-rmse:36.56125	val-rmse:41.46452
[4]	train-rmse:35.44252	val-rmse:40.88896
[5]	train-rmse:34.57756	val-rmse:40.69096
[6]	train-rmse:33.84230	val-rmse:40.59315
[7]	train-rmse:33.25929	val-rmse:40.47993
[8]	train-rmse:32.79415	val-rmse:40.45326
[9]	train-rmse:32.16019	val-rmse:40.43929
[10]	train-rmse:31.63404	val-rmse:40.48319
[11]	train-rmse:31.17673	val-rmse:40.68201
[12]	train-rmse:30.87313	val-rmse:40.63522
[13]	train-rmse:30.30310	val-rmse:40.70983
[14]	train-rmse:30.00098	val-rmse:40.78133
[15]	train-rmse:29.41497	val-rmse:40.86107
[16]	train-rmse:29.25816	val-rmse:40.96580
[17]	train-rmse:28.59378	val-rmse:41.12190
[18]	train-rmse:28.27990	val-rmse:41.14360
[19]	train-rmse:27.94572	val-rmse:41.22835
[20]	train-rmse:27.49658	val-rmse:41.27921
[21]	train-rmse:27.25449	val-rmse:41.32427
[22]	train-rmse:27.06652	val-rmse:41.41887
[23]	train-rmse:26.78

In [124]:
xgbr = xgb.train(
    xgb_params2,
    dtrain,
    num_boost_round = 100,
    evals = watchlist
)

[0]	train-rmse:45.49999	val-rmse:47.00533
[1]	train-rmse:44.12948	val-rmse:45.92344
[2]	train-rmse:42.94858	val-rmse:44.98366
[3]	train-rmse:41.90896	val-rmse:44.25755
[4]	train-rmse:40.96728	val-rmse:43.57339
[5]	train-rmse:40.17514	val-rmse:43.11181
[6]	train-rmse:39.40436	val-rmse:42.61054
[7]	train-rmse:38.71199	val-rmse:42.18883
[8]	train-rmse:38.08081	val-rmse:41.86754
[9]	train-rmse:37.57559	val-rmse:41.64338
[10]	train-rmse:37.07490	val-rmse:41.39235
[11]	train-rmse:36.58709	val-rmse:41.14265
[12]	train-rmse:36.14574	val-rmse:40.95201
[13]	train-rmse:35.76896	val-rmse:40.81778
[14]	train-rmse:35.40802	val-rmse:40.75008
[15]	train-rmse:35.08521	val-rmse:40.61341
[16]	train-rmse:34.74187	val-rmse:40.51800
[17]	train-rmse:34.43969	val-rmse:40.41659
[18]	train-rmse:34.16132	val-rmse:40.33546
[19]	train-rmse:33.91836	val-rmse:40.25632
[20]	train-rmse:33.67389	val-rmse:40.25010
[21]	train-rmse:33.45853	val-rmse:40.19826
[22]	train-rmse:33.23371	val-rmse:40.21101
[23]	train-rmse:32.98